In [1]:
import numpy as np
import pandas as pd
import os
import time
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import ADASYN

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, Bidirectional, Flatten
from keras.models import Sequential
from keras.models import model_from_json

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = pd.read_csv("C:/Users/user/PythonProjects/document-data-extraction/data-extraction/training/train3_dv_lm_iv.csv",header=None)
n=len(dataset.columns)-1

In [3]:
X = dataset.iloc[:,0:n]
Y = dataset.iloc[:,n].values
X, Y = ADASYN().fit_sample(X, Y)

In [5]:
Y1=[]
i=[0,0,0,0,0]
for y in Y:
    if(y=='AIMX'):
        Y1.append(0)
        i[0]=i[0]+1
    if(y=='BASE'):
        Y1.append(1)
        i[1]=i[1]+1
    if(y=='CONT'):
        Y1.append(2)
        i[2]=i[2]+1
    if(y=='MISC'):
        Y1.append(3)
        i[3]=i[3]+1
    if(y=='OWNX'):
        Y1.append(4)
        i[4]=i[4]+1
print(i)

[641, 627, 616, 623, 642]


In [5]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y1, test_size = 0.40, random_state = 8)
print("Training size:", x_train.shape[0])
print("Testing size:", x_test.shape[0])
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

Training size: 1889
Testing size: 1260


In [6]:
x_train1=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_test1=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test)

In [7]:
model=Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Conv1D(100, 50, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()          

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 384, 200)          81600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 384, 200)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 384, 200)          240800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 384, 200)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 335, 100)          1000100   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 67, 100)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6700)              0         
__________

In [68]:
start_time = time.time()
model.fit(x_train1, y_train1, validation_data=(x_test1, y_test1),epochs=10, batch_size=100)
print("Total time to train:",(time.time() - start_time))

Train on 1889 samples, validate on 1260 samples
Epoch 1/10
1889/1889 [==============================] - 56s 30ms/step - loss: 1.5169 - acc: 0.2880 - val_loss: 1.4633 - val_acc: 0.3833
Epoch 2/10
1889/1889 [==============================] - 53s 28ms/step - loss: 1.3203 - acc: 0.4203 - val_loss: 1.2893 - val_acc: 0.4730
Epoch 3/10
1889/1889 [==============================] - 53s 28ms/step - loss: 1.1473 - acc: 0.5013 - val_loss: 1.1203 - val_acc: 0.5341
Epoch 4/10
1889/1889 [==============================] - 52s 28ms/step - loss: 1.0277 - acc: 0.5691 - val_loss: 1.0592 - val_acc: 0.5706
Epoch 5/10
1889/1889 [==============================] - 52s 27ms/step - loss: 0.9209 - acc: 0.6368 - val_loss: 1.0030 - val_acc: 0.5921
Epoch 6/10
1889/1889 [==============================] - 52s 28ms/step - loss: 0.8499 - acc: 0.6713 - val_loss: 1.0135 - val_acc: 0.6190
Epoch 7/10
1889/1889 [==============================] - 53s 28ms/step - loss: 0.8117 - acc: 0.6660 - val_loss: 0.8904 - val_acc: 0.6563


Save Model

In [1]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


Load Model

In [21]:
model_json = model.to_json()
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [22]:
y_pred1 = model.predict(x_test1)
y_pred=np.argmax(y_pred1,axis=1)

In [23]:
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[232   0   0  12   8]
 [  0 259   0   0   0]
 [  0   0 240   5   4]
 [  6   2   2 176  73]
 [  4   6   1  76 154]]


In [24]:
accuracy=accuracy_score(y_pred,y_test)
print(accuracy*100,'%')

84.2063492063492 %
